In [20]:
import pandas as pd
import error_prompts as p
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()



prompt_name = 'error_class_LLM'
prompt = p.error_class_LLM

file = "structured_unstructured_zero_shot_multi_table_gpt-3.5-turbo"

DATA_PATH = f'../../Datasets/Evaluations/Schema Matching/Error_Analysis/{file}.csv'
OUTPUT_PATH = f"../../Datasets/Evaluations/Schema Matching/Error_Analysis/error_class_LLM_analysis.csv"

MODEL = "gpt-3.5-turbo"
TEMP = 0

struct_unstruct = 'unstructured_analysis'

In [21]:
import os
df = pd.read_csv(DATA_PATH)
print(df.shape)
print(df.head())

(25, 17)
   Unnamed: 0  table_index                      table_name  column_index_left  \
0         183          183  36039980_4_4283009829602711082                  1   
1         137          137  86747932_0_7532457067740920052                  3   
2         215          215  86297395_0_6919201319699354263                  2   
3         294          294  39173938_0_7916056990138658530                  4   
4         438          438  90593344_0_8311455501234425088                  2   

   column_index_right  y_true  y_pred            prompt_name  \
0                   4    True   False  zero_shot_multi_table   
1                   0    True   False  zero_shot_multi_table   
2                   2    True   False  zero_shot_multi_table   
3                   0    True   False  zero_shot_multi_table   
4                   2    True   False  zero_shot_multi_table   

                                              prompt  \
0  System: Description: Please identify the match...   
1  Syst

In [22]:
df_correct = df[df['error'] == 'correct']
df_correct_samples = df_correct.sample(n=3, random_state=42)

df_error = df[df['error'] == 'error']
df_error_samples = df_error.sample(n=12, random_state=42)

print(df_correct_samples.shape, df_error_samples.shape)
print(df_correct_samples.head())
print(df_error_samples.head())

if os.path.exists(OUTPUT_PATH):
    df_answer = pd.read_csv(OUTPUT_PATH)
else:
    df_answer = pd.DataFrame(columns=['Prompt_Name', 'Prompt', 'Model', 'Database', 'Answer', 'Context'])

df_correct_samples = df_correct_samples.reset_index(drop=True)
df_error_samples = df_error_samples.reset_index(drop=True)






(3, 17) (12, 17)
    Unnamed: 0  table_index                      table_name  \
21         196          196  48456557_0_3760853481322708783   
24          28           28  55961337_0_6548713781034932742   
22         422          422  49801939_0_6964113429298874283   

    column_index_left  column_index_right  y_true  y_pred  \
21                  2                   1   False   False   
24                  4                   0   False   False   
22                  1                   2    True    True   

              prompt_name                                             prompt  \
21  zero_shot_multi_table  System: Description: Please identify the match...   
24  zero_shot_multi_table  System: Description: Please identify the match...   
22  zero_shot_multi_table  System: Description: Please identify the match...   

                                       column_table_A  \
21  Mephitis mephitis | Orconectes punctimanus | R...   
24                        FKP | DJF | XCD | DKK | 

In [23]:
def execute(df, prompt, df_answer):
    i = 1
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=150)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=150)
    examples = ""
    examples += "Wrong Decisions:\n"
    for j in range(len(df_error_samples)):
        if str(df_error_samples.iloc[j]['y_pred']) == 'True':
            pred_match = "match"
        else:
            pred_match = "do not match"

        if str(df_error_samples.iloc[j]['y_true']) == 'True':
            true_match = "match"
        else:
            true_match = "do not match"
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_error_samples.iloc[j]['prompt'] + "\n"
        examples += "Matching Decisions: " + str(df_error_samples.iloc[j]['ai_answer']) + "\n"
        examples += "Predicted answer for Specific Column Pair: Column A-" + str(df_error_samples.iloc[j]['column_index_left']) + " and Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + pred_match + "\n"
        examples += "Explanation: " + df_error_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Correct answer for Specific Column Pair: Column A-" + str(df_error_samples.iloc[j]['column_index_left']) + " Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + true_match  + "\n\n\n"
        i += 1
    examples += "\nCorrect Decisions (Only as a reference):\n"
    for j in range(len(df_correct_samples)):
        if str(df_correct_samples.iloc[j]['y_pred']) == 'True':
            pred_match = "match"
        else:
            pred_match = "do not match"

        if str(df_correct_samples.iloc[j]['y_true']) == 'True':
            true_match = "match"
        else:
            true_match = "do not match"
        examples += f"Task {i}:\n"
        examples += "Prompt: " + df_correct_samples.iloc[j]['prompt'] + "\n"
        examples += "Matching Decision Output: " + str(df_correct_samples.iloc[j]['ai_answer']) + "\n"
        examples += "Predicted answer for Specific Column Pair: Column A-" + str(df_correct_samples.iloc[j]['column_index_left']) + " and Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + pred_match + "\n"
        examples += "Explanation: " + df_correct_samples.iloc[j][struct_unstruct] + "\n"
        examples += "Correct answer for Specific Column Pair: Column A-" + str(df_error_samples.iloc[j]['column_index_left']) + " Column B-" + str(df_error_samples.iloc[j]['column_index_right']) + " " + true_match  + "\n"
        i += 1
    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
    try: 
        result = chain.run(examples = str(examples), callbacks=[handler])
    except Exception as e:
        print(e)
        print(chain.prompt.format_prompt(examples = str(examples)).to_string())
    handler.langfuse.flush()
    prompt_text = chain.prompt.format_prompt(examples = str(examples)).to_string()
    print(prompt_text)
    print("\n")
    print(result)
    print("\n")
    new_row = {'Prompt_Name': prompt_name, 'Prompt': prompt_text, 'Model': MODEL, 'Database': file, 'Answer': result, 'Context': struct_unstruct}
    df_new_row = pd.DataFrame([new_row])
    df_answer = pd.concat([df_answer, df_new_row])
    return result, df_answer


answer, df_answer = execute(df, prompt, df_answer)
    



In the following I will give you schema matching tasks together with column matching decisions. You are given additional explanation on the predicted matching decision for two specific columns and the actual answer regarding the match of the two columns.
The decision was made by an LLM. Can you please group the wrong decisions into distinct fault categories? Please also indicate how often each one occurs.
There are also some correct decisions in the examples. Please just use them as a reference and don't categorize them.
Wrong Decisions:
Task 1:
Prompt: System: Description: Please identify the matching columns between Table A and Table B. 
     For each column in Table A, specify the corresponding column in Table B. 
     If a column in A has no corresponding column in Table B, you can map it to 'None'. 
     Represent each column mapping using a pair of column headers in a list, i.e., [Table A Column, Table B column or None]. 
     Provide the mapping for each column in Table A and re

In [24]:
print(df_answer)

       Prompt_Name                                             Prompt  \
0  error_class_LLM  In the following I will give you schema matchi...   
0  error_class_LLM  In the following I will give you schema matchi...   

                 Model                                           Database  \
0  gpt-4-turbo-preview  structured_unstructured_zero_shot_multi_table_...   
0        gpt-3.5-turbo  structured_unstructured_zero_shot_multi_table_...   

                                              Answer                Context  
0  Based on the provided examples and explanation...  unstructured_analysis  
0  Fault Categories:\n1. Content Mismatch:\n   - ...  unstructured_analysis  


In [14]:
df_answer.to_csv(OUTPUT_PATH, index=False)